In [1]:
import json
golden_path = './data/small/origin_data/dev_data.json'
predict_path = './out/pred_out'
from config import opt
from helpData import load_data
from metrics import classification_report
from pytorch_pretrained_bert import BertTokenizer

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
def get_positions(data_list, map_str):
    """
    返回实体在单词列表中的位置
    sample:
    >> input: ['球','星','姚'，'明', ...., ], '姚明'
    >> return: (2, 3)
    """
    map_str = map_str.strip().replace(' ', '$')
    map_str = tokenizer_n.tokenize(map_str)
    map_str = [i.replace('#', '') for i in map_str]
    map_str = ''.join(map_str)
    data_list = [i.replace('#', '') for i in data_list]
    # 如果只由一个词组成
    for word in data_list:
        if map_str.lower() in word.lower():
            start_id = end_id = data_list.index(word)
            return start_id, end_id

    start_id = -1
    end_id = -1
    for idx, word in enumerate(data_list):
        if start_id != - 1 and end_id != -1:
            return start_id, end_id
        if map_str.startswith(word):
            start_id = end_id = idx
            while end_id+1 < len(data_list) and data_list[end_id+1] in map_str:
                if "".join(data_list[start_id:end_id+2]) == map_str:
                    # print("".join(data_list[start_id:end_id+3]))
                    return start_id, end_id+1
                end_id += 1
            find_str = ""
            for idx in range(start_id, end_id+1):
                find_str = find_str + data_list[idx]
            if find_str != map_str:
                pre_extend = (data_list[start_id-1] if start_id > 0 else "") + find_str
                last_extend = find_str + (data_list[end_id+1] if end_id < len(data_list)-1 else "")
                pre_last_extend = (data_list[start_id-1] if start_id > 0 else "")+ find_str + (data_list[end_id+1] if end_id < len(data_list)-1 else "")
                if map_str in pre_extend:
                    start_id -= 1
                elif map_str in last_extend:
                    end_id += 1
                elif map_str in pre_last_extend:
                    start_id -= 1
                    end_id += 1
                else:
                    start_id = -1
                    end_id = -1
    if start_id != -1 and end_id != -1:
        return start_id, end_id
    for idx, word in enumerate(data_list[:-1]):
        if map_str in (word+data_list[idx+1]):
            return idx,idx+1
    # print("word_list{}  map_str {} loss".format(data_list, map_str))
    return start_id, end_id

In [3]:
def load_data(path):
    '''
    加载数据，返回json数组.
    '''
    data = []
    data_lines = open(path, encoding='utf-8').readlines()
    for line in data_lines:
        data.append(json.loads(line))
    return data
predict_data = load_data(predict_path)
golden_data = load_data(golden_path)[:len(predict_data)]

In [4]:
def get_relation_num(datas):
    r2num = {}
    for data in datas:
        spo_lists = data['spo_list']
        for spo in spo_lists:
            r = spo['predicate']
            if not r in r2num.keys():
                r2num[r] = 1
            else:
                r2num[r] += 1
    return sorted(r2num.items(), key=lambda d:d[0])
    return r2num

In [5]:
def get_text2spo(datas):
    text2spo = {}
    for data in datas:
        text2spo[data['text']] = data['spo_list']
    return text2spo

In [6]:
def get_right_relaiton_num(golden_data, predict_data):
    golden_data = get_text2spo(golden_data)
    predict_data = get_text2spo(predict_data)
    print(len(golden_data))
    rightr2num = {}
    for text in golden_data.keys():
        true_spos = golden_data[text]
        pred_spos = predict_data.get(text, [])
        all_true_predict = [predict['predicate'] for predict in true_spos]
        for pred in pred_spos:
            if pred['predicate'] in all_true_predict:
                if not pred['predicate'] in rightr2num.keys():
                    rightr2num[pred['predicate']] = 1
                else:
                    rightr2num[pred['predicate']] += 1
    return sorted(rightr2num.items(), key=lambda d:d[0])

In [7]:
right_rnum = get_right_relaiton_num(golden_data, predict_data)

21629


In [9]:
golden_rnum = get_relation_num(golden_data)
predict_rnum = get_relation_num(predict_data)
print("gloden relation num: {} \n".format(golden_rnum))
print("predict relation num: {} \n".format(predict_rnum))
print("predict right num {}".format(right_rnum))

gloden relation num: [('丈夫', 680), ('上映时间', 463), ('专业代码', 3), ('主持人', 208), ('主演', 7411), ('主角', 39), ('人口数量', 45), ('作曲', 1123), ('作者', 4390), ('作词', 1119), ('修业年限', 2), ('出品公司', 851), ('出版社', 2270), ('出生地', 2370), ('出生日期', 2894), ('创始人', 81), ('制片人', 144), ('占地面积', 61), ('号', 197), ('嘉宾', 171), ('国籍', 1654), ('妻子', 680), ('字', 339), ('官方语言', 15), ('导演', 2697), ('总部地点', 310), ('成立日期', 1606), ('所在城市', 77), ('所属专辑', 1287), ('改编自', 71), ('朝代', 419), ('歌手', 3085), ('母亲', 425), ('毕业院校', 1433), ('民族', 1436), ('气候', 102), ('注册资本', 63), ('海拔', 43), ('父亲', 609), ('目', 1271), ('祖籍', 181), ('简称', 138), ('编剧', 866), ('董事长', 50), ('身高', 392), ('连载网站', 1657), ('邮政编码', 4), ('面积', 79), ('首都', 47)] 

predict relation num: [('丈夫', 942), ('上映时间', 378), ('专业代码', 1), ('主持人', 209), ('主演', 9253), ('主角', 30), ('人口数量', 49), ('作曲', 936), ('作者', 4737), ('作词', 1021), ('修业年限', 1), ('出品公司', 966), ('出版社', 2463), ('出生地', 2460), ('出生日期', 3079), ('创始人', 90), ('制片人', 167), ('占地面积', 65), ('号', 260), ('嘉宾', 284), ('国籍',

In [10]:
def get_case_ans(golden_data, predict_data):
    predict_data = get_text2spo(predict_data)
    golden_data = get_text2spo(golden_data)
    case = {}
    for text in golden_data.keys():
        true_spos = golden_data[text]
        for true in true_spos:
            true['object_type'], true['subject_type'] = None, None
            true['object'], true['subject'] = true['object'].lower(), true['subject'].lower()
            
        pred_spos = predict_data.get(text, [])
        for pred in pred_spos:
            pred['object_type'], pred['subject_type'] = None, None
        unit = {'right':[], "loss":[], 'wrong':[]}
        for true in true_spos:
            if not true in pred_spos:
                unit['loss'].append(true)
            if true in pred_spos:
                unit['right'].append(true)
        for pred in pred_spos:
            if not pred in true_spos:
                unit['wrong'].append(pred)
        if len(unit['loss']) == 0 and len(unit['wrong']) == 0:
            continue
        case[text] = unit
    return case

In [11]:
case = get_case_ans(golden_data, predict_data)

In [12]:
tokenizer_n = BertTokenizer.from_pretrained(opt.bert_vocab_unk, do_lower_case=True)
tokenizer_o = BertTokenizer.from_pretrained(opt.bert_vocab_dir, do_lower_case=True)

In [13]:
wrong_case = []
for k in case.keys():
    c = case[k]
    if len(c['loss']) != 0 or len(c['wrong']) != 0:
        wrong_case.append((k, c))

In [17]:
wrong_case[-100:]

[('《别让我再见到你》是1999年陈洁仪演唱的一首歌曲，所属专辑是《炫耀》',
  {'right': [{'predicate': '歌手',
     'object_type': None,
     'subject_type': None,
     'object': '陈洁仪',
     'subject': '别让我再见到你'},
    {'predicate': '所属专辑',
     'object_type': None,
     'subject_type': None,
     'object': '炫耀',
     'subject': '别让我再见到你'}],
   'loss': [{'predicate': '歌手',
     'object_type': None,
     'subject_type': None,
     'object': '陈洁仪',
     'subject': '炫耀'}],
   'wrong': []}),
 ('长城汽车是长城汽车股份有限公司的简称，长城汽车的前身是长城工业公司，是一家集体所有制企业，成立于1984年，公司总部位于河北省保定市',
  {'right': [{'predicate': '成立日期',
     'object_type': None,
     'subject_type': None,
     'object': '1984年',
     'subject': '长城汽车'}],
   'loss': [{'predicate': '总部地点',
     'object_type': None,
     'subject_type': None,
     'object': '保定市',
     'subject': '长城汽车'}],
   'wrong': [{'object': '河北省保定市',
     'subject': '长城汽车',
     'predicate': '总部地点',
     'object_type': None,
     'subject_type': None}]}),
 ('《节能省地型住宅和公共建筑标准规范汇编（上下）》是2005年中国建筑工业出版社出版的图书，作者是建设部标准定额研

In [14]:
text = '《下班抓紧谈恋爱》是万达影视传媒有限公司、上海强胜影视文化传媒有限公司联合出品的的都市情感剧，由万洋执导，朱雨辰、杜海涛、徐黄丽、杨淇等主演1'
text = text.strip().replace(' ', '$')
word_list = tokenizer_n.tokenize(text)

In [15]:
get_positions(word_list, '万达影视传媒有限公司')

(10, 19)

In [20]:
word_list[10:20]

['万', '达', '影', '视', '传', '媒', '有', '限', '公', '司']